
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.



In [1]:
"""
WITH
     inv_2011 AS (SELECT country_code, AVG(funding_total) AS tot
                 FROM company
                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2011
                 GROUP BY country_code
                 ),

     inv_2012 AS (SELECT country_code, AVG(funding_total) AS tot
                 FROM company
                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2012
                 GROUP BY country_code
                 ),
     inv_2013 AS (SELECT country_code, AVG(funding_total) AS tot
                 FROM company
                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2013
                 GROUP BY country_code
                 )

SELECT inv_2011.country_code, inv_2011.tot AS i_11, inv_2012.tot AS i_12, inv_2013.tot AS i_13

FROM inv_2011 JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code JOIN inv_2013 ON inv_2012.country_code = inv_2013.country_code

ORDER BY inv_2011.tot DESC

"""

'\nWITH\n     inv_2011 AS (SELECT country_code, AVG(funding_total) AS tot\n                 FROM company\n                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2011\n                 GROUP BY country_code\n                 ),  \n                 \n     inv_2012 AS (SELECT country_code, AVG(funding_total) AS tot\n                 FROM company\n                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2012\n                 GROUP BY country_code\n                 ),\n     inv_2013 AS (SELECT country_code, AVG(funding_total) AS tot\n                 FROM company\n                 WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2013\n                 GROUP BY country_code\n                 )\n                 \nSELECT inv_2011.country_code, inv_2011.tot AS i_11, inv_2012.tot AS i_12, inv_2013.tot AS i_13\n      \nFROM inv_2011 JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code JOIN inv_2013 ON inv_2012.country_code = inv_2013.country_code \n\nORDER

Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
"""
WITH
d AS (SELECT a.id AS id, c.name AS name, a.price_amount AS price
FROM acquisition AS a LEFT OUTER JOIN company AS c ON c.id = a.acquiring_company_id),

b AS (SELECT ac.id AS id_s, co.name AS name_s, co.funding_total AS funding
FROM acquisition AS ac LEFT OUTER JOIN company AS co ON co.id = ac.acquired_company_id)

SELECT d.name, d.price, b.name_s, b.funding, ROUND(d.price/b.funding)
FROM d JOIN b ON d.id = b.id_s
WHERE b.funding != 0
ORDER BY d.price DESC, b.name_s
LIMIT 10
"""

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
"""
WITH
a AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS DATE)) AS month, COUNT(acquired_company_id) AS comp, SUM(price_amount) AS price
FROM acquisition
WHERE EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) >= 2010
GROUP BY month),

b AS (SELECT EXTRACT(MONTH FROM CAST(funded_at AS DATE)) AS month, COUNT(DISTINCT f.name) AS names
FROM investment AS i JOIN funding_round AS fr ON i.funding_round_id = fr.id JOIN fund AS f ON f.id = i.fund_id
WHERE f.country_code = 'USA' AND EXTRACT(YEAR FROM CAST(funded_at AS DATE)) >= 2010
GROUP BY month)

SELECT a.month, b.names, a.comp, a.price
      FROM a JOIN b ON a.month=b.month
"""

Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [3]:
"""
WITH views AS (SELECT DISTINCT CAST(DATE_TRUNC('month', creation_date) AS DATE), SUM(views_count) OVER(PARTITION BY CAST(DATE_TRUNC('month', creation_date) AS DATE)) AS sum_views
FROM stackoverflow.posts
WHERE CAST(DATE_TRUNC('year', creation_date) AS DATE) = '2008-01-01')

SELECT *
FROM views
ORDER BY sum_views DESC

"""

"\nWITH views AS (SELECT DISTINCT CAST(DATE_TRUNC('month', creation_date) AS DATE), SUM(views_count) OVER(PARTITION BY CAST(DATE_TRUNC('month', creation_date) AS DATE)) AS sum_views\nFROM stackoverflow.posts\nWHERE CAST(DATE_TRUNC('year', creation_date) AS DATE) = '2008-01-01')\n\nSELECT *\nFROM views\nORDER BY sum_views DESC\n\n"

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [5]:
"""
WITH users AS (SELECT DISTINCT user_id
FROM stackoverflow.posts p JOIN stackoverflow.users u ON p.user_id = u.id
WHERE CAST(DATE_TRUNC('month', u.creation_date) AS DATE) = '2008-09-01' AND CAST(DATE_TRUNC('month', p.creation_date) AS DATE) = '2008-12-01')

SELECT DISTINCT CAST(DATE_TRUNC('month', p.creation_date) AS DATE), COUNT(*) OVER (PARTITION BY CAST(DATE_TRUNC('month', p.creation_date) AS DATE))
FROM users u JOIN stackoverflow.posts p ON u.user_id = p.user_id
WHERE CAST(DATE_TRUNC('year', p.creation_date) AS DATE) = '2008-01-01'
ORDER BY CAST(DATE_TRUNC('month', p.creation_date) AS DATE) DESC
"""

"\nWITH users AS (SELECT DISTINCT user_id \nFROM stackoverflow.posts p JOIN stackoverflow.users u ON p.user_id = u.id\nWHERE CAST(DATE_TRUNC('month', u.creation_date) AS DATE) = '2008-09-01' AND CAST(DATE_TRUNC('month', p.creation_date) AS DATE) = '2008-12-01')\n\nSELECT DISTINCT CAST(DATE_TRUNC('month', p.creation_date) AS DATE), COUNT(*) OVER (PARTITION BY CAST(DATE_TRUNC('month', p.creation_date) AS DATE))\nFROM users u JOIN stackoverflow.posts p ON u.user_id = p.user_id\nWHERE CAST(DATE_TRUNC('year', p.creation_date) AS DATE) = '2008-01-01'\nORDER BY CAST(DATE_TRUNC('month', p.creation_date) AS DATE) DESC\n"


Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [6]:
"""
WITH users AS (SELECT DISTINCT user_id, CAST(creation_date AS DATE)
FROM stackoverflow.posts
WHERE CAST(creation_date AS DATE) >= '2008-12-01' AND CAST(creation_date AS DATE) <= '2008-12-07'
             ),

users_days AS (SELECT COUNT(*) AS cnt
FROM users
GROUP BY user_id)

SELECT ROUND(AVG(cnt))
FROM users_days

"""

"\nWITH users AS (SELECT DISTINCT user_id, CAST(creation_date AS DATE)\nFROM stackoverflow.posts\nWHERE CAST(creation_date AS DATE) >= '2008-12-01' AND CAST(creation_date AS DATE) <= '2008-12-07'\n             ),\n\nusers_days AS (SELECT COUNT(*) AS cnt\nFROM users\nGROUP BY user_id)\n\nSELECT ROUND(AVG(cnt))\nFROM users_days\n\n"


На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
Номер месяца.
Количество постов за месяц.
Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

In [7]:
"""
WITH month_cnt AS

(SELECT DISTINCT EXTRACT(MONTH FROM creation_date) AS mnth, COUNT(*) OVER(PARTITION BY CAST(DATE_TRUNC('month', creation_date) AS DATE)) AS cnt
FROM stackoverflow.posts
WHERE CAST(creation_date AS DATE) >= '2008-09-01' AND CAST(creation_date AS DATE) <= '2008-12-31'
ORDER BY EXTRACT(MONTH FROM creation_date))

SELECT *, ROUND( 100 * ((cnt :: numeric / LAG(cnt) OVER(ORDER BY mnth)) :: numeric - 1), 2)  AS changing
FROM month_cnt
"""

"\nWITH month_cnt AS \n\n(SELECT DISTINCT EXTRACT(MONTH FROM creation_date) AS mnth, COUNT(*) OVER(PARTITION BY CAST(DATE_TRUNC('month', creation_date) AS DATE)) AS cnt\nFROM stackoverflow.posts\nWHERE CAST(creation_date AS DATE) >= '2008-09-01' AND CAST(creation_date AS DATE) <= '2008-12-31'\nORDER BY EXTRACT(MONTH FROM creation_date))\n\nSELECT *, ROUND( 100 * ((cnt :: numeric / LAG(cnt) OVER(ORDER BY mnth)) :: numeric - 1), 2)  AS changing\nFROM month_cnt\n"

Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [8]:
"""
WITH user_max AS
(
SELECT user_id, COUNT(id) AS cnt
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY COUNT(id) DESC
LIMIT 1
    )

SELECT DISTINCT EXTRACT('week' FROM p.creation_date), MAX(p.creation_date) OVER(PARTITION BY EXTRACT('week' FROM p.creation_date))
FROM stackoverflow.posts p JOIN user_max u ON u.user_id = p.user_id
WHERE CAST(DATE_TRUNC('month', p.creation_date) AS DATE) = '2008-10-01'
"""

"\nWITH user_max AS\n(\nSELECT user_id, COUNT(id) AS cnt\nFROM stackoverflow.posts\nGROUP BY user_id\nORDER BY COUNT(id) DESC\nLIMIT 1\n    )\n    \nSELECT DISTINCT EXTRACT('week' FROM p.creation_date), MAX(p.creation_date) OVER(PARTITION BY EXTRACT('week' FROM p.creation_date))\nFROM stackoverflow.posts p JOIN user_max u ON u.user_id = p.user_id\nWHERE CAST(DATE_TRUNC('month', p.creation_date) AS DATE) = '2008-10-01'\n"


Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [9]:
"""
WITH users AS(SELECT id, views,
CASE
    WHEN views >= 350 THEN 1
    WHEN views < 350 AND views >= 100 THEN 2
    WHEN views < 100 THEN 3
END views_group
FROM stackoverflow.users
WHERE location LIKE '%Canada%'),

users_views AS (SELECT *
FROM users
WHERE views > 0),

maxs AS (SELECT *, MAX(views) OVER(PARTITION BY views_group) AS max_views
FROM users_views)

SELECT id, views_group, views
FROM maxs
WHERE max_views = views
ORDER BY views DESC, id
"""

"\nWITH users AS(SELECT id, views, \nCASE\n    WHEN views >= 350 THEN 1\n    WHEN views < 350 AND views >= 100 THEN 2\n    WHEN views < 100 THEN 3 \nEND views_group\nFROM stackoverflow.users\nWHERE location LIKE '%Canada%'), \n\nusers_views AS (SELECT *\nFROM users\nWHERE views > 0), \n\nmaxs AS (SELECT *, MAX(views) OVER(PARTITION BY views_group) AS max_views\nFROM users_views)\n\nSELECT id, views_group, views\nFROM maxs\nWHERE max_views = views\nORDER BY views DESC, id\n"

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [11]:
"""
ITH users_badges AS(
SELECT DISTINCT u.id AS u_id, COUNT(b.id) OVER(PARTITION BY u.id) AS cnt_badges
FROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE CAST(b.creation_date AS DATE) >= '15-11-2008' AND CAST(b.creation_date AS DATE) <= '15-12-2008')

SELECT *, DENSE_RANK() OVER(ORDER BY cnt_badges DESC)
FROM users_badges
ORDER BY cnt_badges DESC, u_id
LIMIT 10
"""

"\nITH users_badges AS(\nSELECT DISTINCT u.id AS u_id, COUNT(b.id) OVER(PARTITION BY u.id) AS cnt_badges\nFROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id = b.user_id\nWHERE CAST(b.creation_date AS DATE) >= '15-11-2008' AND CAST(b.creation_date AS DATE) <= '15-12-2008')\n\nSELECT *, DENSE_RANK() OVER(ORDER BY cnt_badges DESC)\nFROM users_badges\nORDER BY cnt_badges DESC, u_id \nLIMIT 10\n"

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [12]:
"""

WITH users AS(
SELECT DISTINCT u.id, u.creation_date, MIN(p.creation_date) OVER(PARTITION BY p.user_id) AS first_post
FROM stackoverflow.users u JOIN stackoverflow.posts p ON u.id = p.user_id)

SELECT id, first_post - creation_date AS gap
FROM users

"""

'\n\nWITH users AS(\nSELECT DISTINCT u.id, u.creation_date, MIN(p.creation_date) OVER(PARTITION BY p.user_id) AS first_post\nFROM stackoverflow.users u JOIN stackoverflow.posts p ON u.id = p.user_id)\n\nSELECT id, first_post - creation_date AS gap\nFROM users\n\n'